In [9]:
import pandas as pd 
import psycopg2

In [ ]:
%load_ext sql
%sql postgresql+psycopg2://postgres-readonly:ZY>[#4Bmx0AFDOkE@etl-relational-database.cfmnnswnfhpn.eu-west-2.rds.amazonaws.com/postgres


In [ ]:
%sql select facid as "Facility ID", COUNT(starttime) as "Total Bookings" from bookings group by facid order by facid limit 1000


In [13]:
conn_params = {
    'host': 'etl-relational-database.cfmnnswnfhpn.eu-west-2.rds.amazonaws.com',
    'dbname': 'postgres',
    'user': 'postgres-readonly',
    'password': 'ZY>[#4Bmx0AFDOkE'
}

analytical_database_conn_params = {
    'host': 'localhost',
    'dbname': 'etl_bites',
    'user': '',
    'password': ''

}

In [14]:
def fetch_data(query):
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            data = cur.fetchall()
            colnames = [desc[0] for desc in cur.description]
    return data, colnames


def execute_query_postgresql(query):
    with psycopg2.connect(**analytical_database_conn_params) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()


In [ ]:
# Extract bookings data
query = "SELECT facid FROM bookings;"
bookings_data, colnames = fetch_data(query)
bookings_df = pd.DataFrame(bookings_data, columns=colnames)

# Transform data: Count bookings per facility
booking_counts_per_facility = bookings_df.groupby('facid').size().reset_index(name='TotalBookings')

# Load the results into your destination database for reporting.
# Define the table structure for the destination database
create_booking_counts_table = '''
CREATE TABLE IF NOT EXISTS booking_counts_per_facility (
    FacilityID INTEGER NOT NULL,
    TotalBookings INTEGER NOT NULL
);
'''

execute_query_postgresql(create_booking_counts_table)

def insert_data_to_postgresql(data, connection):
    cursor = connection.cursor()
    query = "INSERT INTO booking_counts_per_facility (FacilityID, TotalBookings) VALUES (%s, %s)"
    for facilityid, row in booking_counts_per_facility.iterrows():
        total_booking = int(row['TotalBookings'])
        cursor.execute(query, (facilityid, total_booking))    
    connection.commit()

etl_bites_conn_string = "host='localhost' port='5432' dbname='etl_bites' user='' password=''"
conn = psycopg2.connect(etl_bites_conn_string)

insert_data_to_postgresql((booking_counts_per_facility), conn)

